In [1]:
import pandas as pd
import csv
import numpy as np
from openai import OpenAI
import os
from bs4 import BeautifulSoup
import requests
from datetime import datetime
current_time = datetime.now().time()
from PIL import Image
import pytesseract
import requests
from io import BytesIO
import cv2
import sys
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# from ..scripts.translation import translate
# from ..scripts.text_embedding import create_text_embeddings


sys.path.append('../scripts')  
from text_embedding import *
from translation import *
from df_handling import *

In [2]:
# #preprocessing steps 

# df4  = pd.read_csv('test1.csv')

# df4['text_embedding'] = df4['text_embedding'].apply(lambda x: x.replace("[","").replace("]",""))

# df4['text_embedding'] = df4['text_embedding'].apply(lambda x: np.array(x.split(',')).astype('float'))

df_postgres = pd.read_csv('../../dfs/postgres.csv')

df = pd.DataFrame(columns=['id','claim','article_title','author','publish_date','harvest_date','url','label', 'article_text'])


In [3]:
#Harvest method for greekhoaxes.gr
def harvest(urls, df):
    for url in urls:
        print(url)
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'html.parser')
        
        conclusion_string = ''
        article_text=''

        #harvest the date
        date = soup.find('div', class_ = 'blog-date').text

        #harvest the claim
        try:
            claim = soup.find('div', class_ = 'Claim')
            claim_text = claim.find_next('p').text

        except AttributeError as e:

            print(f"Attribute Error: {e}")
            continue
    
        
        #harvest the title
        title = soup.find('div', class_ = 'content-bottom')
        title_text = title.find_next('h1').get_text()

        #harvest the author 
        author = soup.find('a', class_ = 'author')
        author_text = author.get_text()

        # harvest the conclusion
        conclusion = soup.find('div', class_='conclusion')
        con_text = conclusion.find_next('p').get_text()

#         print(f'''Date: {date} 
# Claim: {claim_text}
# Title: {title_text}
# Author: {author_text}
# Conclusion: {con_text}
# URL: {url}''')

        #harvest the article text
        article = soup.find('h3',class_='wp-block-heading')
        if(article is None):
            article = soup.find('h4',class_='wp-block-heading')
            if(article is None):
                article = soup.find('h5',class_='wp-block-heading')
                if(article is None):
                    article = soup.find('h2',class_='wp-block-heading')
        try:
            article_text = article.find_next_siblings('p')
        except AttributeError as e:
            print(f"AttributeError: {e}")
        #print('Content:')
        for text in article_text:
            content = text.get_text().strip()
            if content:
                # print(content)
                conclusion_string += content


        #add the harvested data to the dataframe

        data = {'id': len(df), 'claim': claim_text, 'article_title': title_text, 'author': author_text, 'publish_date': date, 
                'harvest_date':current_time , 'url': url, 'article_text': conclusion_string, 'label': 0 }
        
        df.loc[len(df)] = data



#         print()
      
        

In [4]:
#Harvesting greekhoaxes.gr
page_numbers = 10
pages = list()

html_text = requests.get('https://www.ellinikahoaxes.gr/category/kathgories/fake-news/').text
soup = BeautifulSoup(html_text, 'html.parser')
urls = soup.find_all('a', class_="btn btn-secondary")


#find the first page
current_page = soup.find('span', class_='page-numbers current')
print(f"Collecting urls from page1: https://www.ellinikahoaxes.gr/category/kathgories/fake-news/")
url_list = [url.get('href') for url in urls]
harvest(url_list, df)

#find the next page to harvest
current_page = soup.find('a', class_='page-numbers')


#find next pages to harvest depending on the page number variable
#after the page #2, we follow the same procedure recurcively
for i in range(page_numbers):

    
    print(f"Collecting urls from page: {current_page.get('href')}")
   
    
    #find next page
    html_text2 = requests.get(current_page.get('href')).text
    soup = BeautifulSoup(html_text2, 'html.parser')
    
    #recursively find its the next page each time
    current_page = soup.find('span', class_='page-numbers current')
    next_page = current_page.find_next('a', class_ = 'page-numbers')
    current_page = next_page
  
    urls = soup.find_all('a', class_="btn btn-secondary")
  
    url_list = [url.get('href') for url in urls]
    harvest(url_list, df)

    # for url in url_list:
    #     print(url)




https://www.ellinikahoaxes.gr/2024/01/14/o-ypourgos-amynas-ton-ipa-lloyd-austin-den-skotothike-sto-kievo/
https://www.ellinikahoaxes.gr/2023/12/31/ochi-o-ypourgos-exoterikon-tis-politeias-tis-tzortzia-den-synelifthi-gia-prodosia/
https://www.ellinikahoaxes.gr/2023/12/22/ochi-o-tzavent-aslam-den-kalese-ellines-polites-na-fygoun-apo-ti-chora/
https://www.ellinikahoaxes.gr/2023/12/22/ochi-o-volontimir-zelenski-den-pairnei-amerikaniki-ypikootita/
https://www.ellinikahoaxes.gr/2023/12/14/ochi-o-soros-den-agorase-ektasi-stin-oukrania-gia-na-tin-kanei-chomateri-radienergon-kai-chimikon-apovliton/
https://www.ellinikahoaxes.gr/2023/12/12/psevdes-oti-o-oukranos-proedros-zelenski-agorase-dyo-giot-axias-75ek-dolarion/
https://www.ellinikahoaxes.gr/2023/12/09/paraskiniaka-plana-tainias-parousiazontai-psevdos-os-prospatheia-propagandas-ton-palaistinion/
https://www.ellinikahoaxes.gr/2023/11/17/gaza-parliament-demolished-video-false/
https://www.ellinikahoaxes.gr/2023/11/14/pakistani-antikythera-500

KeyboardInterrupt: 

In [3]:
def harvest_2(urls, df):
    for url in urls:
        print(f"Harvesing page: {url}")
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'html.parser')
        article_text = ''
        author_text = ''
        title = soup.find('h1', class_ = 'content-title').text.strip()
        date = soup.find('span', class_ = 'date').text
        authors = soup.find('span', class_ = 'meta-author')
        image_url = soup.find('div', class_='responsive-background-image').find('img', src=True).get('src')

        for author in authors:
            if(author_text!=''):
                author_text = author_text + author.text 
            else:
                author_text = author.text 

            

        article = soup.find_all('div', class_ = 'article-entry clearfix')
        for p in article:
            article_text += p.text #.replace("'","")


        # image = Image.open(requests.get(image_url, stream=True).raw)
        # #image_text = pytesseract.image_to_string(image,lang='Greek')
        # roi_coords = (199, 250, 772, 629)
        # roi_image = image.crop(roi_coords)
        # image_text = pytesseract.image_to_string(roi_image,lang='ell')

            
          



#         print(f'''
# Title: {title}
# Date: {date}
# Authors: {author_text}
# Claim: {title}
# Main_article: {article_text}
# Image url: {image_url}
# Image text: {image_text}

# ''')
    #print(article_text)
        

        data = {'id': len(df), 'claim': title, 'article_title': title, 'author': author_text, 'publish_date': date, 
                    'harvest_date':current_time , 'url': url, 'article_text': str(article_text).strip(), 'label': 1 }
    
        df.loc[len(df)] = data

In [13]:


URL = 'https://factcheckgreek.afp.com/list?page='

url_list = list()
#we iterate through all the pages of the site
for i in range(22):
    
    html_text = requests.get(f"https://factcheckgreek.afp.com/list?page={i}").text
    soup = BeautifulSoup(html_text, 'html.parser')

    titles = soup.find_all('h2', class_='h4 card-title')
    urls = soup.find_all('div', class_='card')
    # for t in titles:
    #     print(t.text)
    for url in urls:
        #print('https://factcheckgreek.afp.com'+url.find_next('a').get('href'))
        url_list.append('https://factcheckgreek.afp.com'+url.find_next('a').get('href'))

harvest_2(url_list,df)
      













Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.34EM7AE
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.34CU2NB
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.34BA7GP
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.349D63B
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.349827A
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.347U4RZ
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.347C36V
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.347F4Q4
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.347A3PY
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.346364R
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.346C3L2
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.344L36H
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.3446798
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.344743V
Harvesing page: https://factcheckgreek.afp.com/doc.afp.com.343

In [14]:

df.shape

(526, 9)

In [9]:
# #harvest for fullfacts.org
def harvest_3(urls,df):
    for url in urls:
        print(url)
        article_text=''
        html_text = requests.get(url, timeout=2000).text
        soup = BeautifulSoup(html_text, 'html.parser')
        title = soup.find('h1', class_='mb-3 highlight-js').text
        try:
            #harvest the variables needed
            claim = soup.find('p', class_='card-text').text
            date = soup.find('div', class_='published-at mb-2').text
            author = soup.find('cite').text
            article = soup.find_all('div', class_='cms-content')
            
            
            for text in article:
                article_text+=text.text.strip()

            #translate article to greek
            article_text = translate(article_text).strip().replace("'","")

            #harvest label
            label = soup.select_one('p.m-0 > a:nth-of-type(2)').text
            print(label)
            if not label :
                label = 'True'

            data = {'id': len(df), 'claim': translate(claim), 'article_title':  translate(title), 'author':  author, 'publish_date': date, 
                    'harvest_date':current_time , 'url': url, 'article_text': article_text, 'label': translate(label) }
            df.loc[len(df)] = data


        
        except TypeError as e:
            print(f"Type Error: {e}")
            article_text = ''

        except AttributeError as e:
            print(f"AttributeError: {e}")
            continue
       
        
        #label = soup.find_all('p', class_='m-0')
        
            

        #     print(f'''
        #  Title: {title}
        #  Claim: {claim}
        #  Date: {date}
        #  Author: {author}
        #  Label: {label}
        #  Text: {article_text}''')







       
        
        

        


        
        


In [61]:
df = pd.DataFrame(columns=['id','claim','article_title','author','publish_date','harvest_date','url','label', 'article_text'])

In [35]:
# #Fullfact.org harvest

static_url = 'https://fullfact.org/latest/?page='
main_url = 'https://fullfact.org'
idx=0
pages = 10 #20/63
batches= 17
url_list = list()
for j in range(batches):
    print(f"Batch no:{j}")
    for i in range(idx,pages):
        print('Page num:', i)
        
        url = static_url + str(i+1)

        print(f"Harvesting url: {url}")
        
        html_text = requests.get(url, timeout=2000).text
        soup = BeautifulSoup(html_text,'html.parser')
        sub_urls = soup.find_all('a', class_ = 'card-link')
        print('Sub-urls:')
        for url in sub_urls:
            # print(main_url+url.get('href'))
            url_list.append(main_url+url.get('href'))
            # harvest_3(main_url+url.get('href'),df)
    #harvest_3(url_list,df)

    pages+=10
    idx+=10
harvest_3(url_list,df)

Batch no:0
Page num: 0
Harvesting url: https://fullfact.org/latest/?page=1
Sub-urls:
Page num: 1
Harvesting url: https://fullfact.org/latest/?page=2
Sub-urls:
Page num: 2
Harvesting url: https://fullfact.org/latest/?page=3
Sub-urls:
Page num: 3
Harvesting url: https://fullfact.org/latest/?page=4
Sub-urls:
Page num: 4
Harvesting url: https://fullfact.org/latest/?page=5
Sub-urls:
Page num: 5
Harvesting url: https://fullfact.org/latest/?page=6
Sub-urls:
Page num: 6
Harvesting url: https://fullfact.org/latest/?page=7
Sub-urls:
Page num: 7
Harvesting url: https://fullfact.org/latest/?page=8
Sub-urls:
Page num: 8
Harvesting url: https://fullfact.org/latest/?page=9
Sub-urls:
Page num: 9
Harvesting url: https://fullfact.org/latest/?page=10
Sub-urls:
Batch no:1
Page num: 10
Harvesting url: https://fullfact.org/latest/?page=11
Sub-urls:
Page num: 11
Harvesting url: https://fullfact.org/latest/?page=12
Sub-urls:
Page num: 12
Harvesting url: https://fullfact.org/latest/?page=13
Sub-urls:
Page num:

In [24]:
df.head(1)

,id,claim,article_title,author,publish_date,harvest_date,url,label,article_text
0,0,"(Έ, ν, α, , β, ί, ν, τ, ε, ο, , δ, ε, ί, χ, ...","(Τ, ο, , β, ί, ν, τ, ε, ο, , δ, ε, ν, , δ, ...",Evie Townend,19 January 2024,20:31:45.524014,https://fullfact.org/news/video-false-claims-y...,"(ψ, ε, υ, δ, ή, ς)","(Ο, ι, , θ, έ, σ, ε, ι, ς, , π, ο, υ, , κ, ..."


In [5]:
df2 = pd.DataFrame(columns=['id','claim','article_title','author','publish_date','harvest_date','url','label', 'article_text'])

In [6]:
#harvest pages for snopes
main_url = 'https://www.snopes.com/fact-check/?pagenum='
pages = 200
url_list = list()

for i in range(1,pages+1):
    html_text = requests.get(f'https://www.snopes.com/fact-check/?pagenum={i}', timeout=100).text
    soup = BeautifulSoup(html_text,'html.parser')
    urls = soup.find_all('a', class_="outer_article_link_wrapper")
    
    for url in urls:
        url_list.append(url.get('href'))
    

In [7]:
#harvest data for snopes.com
for url in url_list:
    print(f"Harvesting url: {url}")
    html_text = requests.get(url, timeout=100).text
    soup = BeautifulSoup(html_text,'html.parser')
    article_text = ''
    #get title
    title = soup.find('section', class_='title-container')
    title_text = title.find('h1').text
   
    
    try:
        #get claim
        claim = soup.find('div', class_='claim_cont').text.strip()
        #get label
        label = soup.find('div', class_='rating_title_wrap').text.strip()
        label_text = label.replace("About this rating","").strip()
    except AttributeError as e:
        print(f"AttributeError: {e}")
        continue
    #get author
    author = soup.find('h3', class_='author_name').text.strip()
    #get date
    date = soup.find('h3', class_='publish_date').text.replace('Published ','')
    #get article
    article = soup.find_all('article',id='article-content' )
    for ar in article:
        article_text+=ar.text.replace("\t","").strip()

    #translation to greek
    try:
        title_text = translate(title_text)
        article_text = translate(article_text)
        claim = translate(claim)
    except requests.exceptions.HTTPError as errh:
        continue
    except

    #if the translation was unsuccessful
    if( (not title_text) | (not article_text) | (not claim) ):
        continue
    else:

        data = {'id': len(df2), 'claim': claim, 'article_title':  title_text, 'author':  author, 'publish_date': date, 
                'harvest_date':current_time , 'url': url, 'article_text': article_text, 'label': label_text}
        df2.loc[len(df2)] = data
    
        


Harvesting url: https://www.snopes.com/fact-check/holes-pasta-spoons/
Harvesting url: https://www.snopes.com/fact-check/belichick-chick-fil-a/
Harvesting url: https://www.snopes.com/fact-check/miami-plane-fire-after-takeoff-video/
Harvesting url: https://www.snopes.com/fact-check/barron-trump-6-foot-7-inches-image/
Harvesting url: https://www.snopes.com/fact-check/russian-conservative-village/
Harvesting url: https://www.snopes.com/fact-check/trump-cassius-clay-spar/
Harvesting url: https://www.snopes.com/fact-check/meryl-streep-at-beatles-concert/
Harvesting url: https://www.snopes.com/fact-check/spotify-locking-christmas-music/
Harvesting url: https://www.snopes.com/fact-check/taylor-swift-trump-shirt/
Harvesting url: https://www.snopes.com/fact-check/pyjama-bird-real/
Harvesting url: https://www.snopes.com/fact-check/trump-investigation-meme/
Harvesting url: https://www.snopes.com/fact-check/free-disney-black-history-month/
Harvesting url: https://www.snopes.com/fact-check/michael-m

HTTPError: HTTP Error 403: Forbidden

In [13]:
harvest_2(df2,['https://www.snopes.com/fact-check/airbnb-in-italy/'])

TypeError: harvest_2() missing 1 required positional argument: 'df'

In [39]:
df.shape

(1451, 9)

In [40]:
df.head()

,id,claim,article_title,author,publish_date,harvest_date,url,label,article_text
0,0,"(Έ, ν, α, , β, ί, ν, τ, ε, ο, , δ, ε, ί, χ, ...","(Τ, ο, , β, ί, ν, τ, ε, ο, , δ, ε, ν, , δ, ...",Evie Townend,19 January 2024,20:31:45.524014,https://fullfact.org/news/video-false-claims-y...,"(ψ, ε, υ, δ, ή, ς)","(Ο, ι, , θ, έ, σ, ε, ι, ς, , π, ο, υ, , κ, ..."
1,1,"(Τ, ο, , A, r, g, o, s, , π, ω, λ, ε, ί, , ...","(Τ, ο, , A, r, g, o, s, , δ, ε, ν, , π, ω, ...",Nasim Asl,17 January 2024,20:31:45.524014,https://fullfact.org/online/argos-playstation-...,"(ψ, ε, υ, δ, ή, ς)","(Ο, ι, , δ, η, μ, ο, σ, ι, ε, ύ, σ, ε, ι, ς, ..."
2,2,"(Μ, ι, α, , α, ν, α, φ, ο, ρ, ά, , π, ο, υ, ...","(Η, , α, ν, α, φ, ο, ρ, ά, , π, ο, υ, , ζ, ...",Charlotte Browning,17 January 2024,20:31:45.524014,https://fullfact.org/online/petition-israel-uk...,"(Λ, ε, ί, π, ε, ι, , τ, ο, , π, λ, α, ί, σ, ...","(Ο, ι, , δ, η, μ, ο, σ, ι, ε, ύ, σ, ε, ι, ς, ..."
3,3,"(Μ, ι, α, , σ, ή, ρ, α, γ, γ, α, , π, ο, υ, ...","(Η, , σ, ή, ρ, α, γ, γ, α, , σ, τ, η, , Σ, ...",Evie Townend,17 January 2024,20:31:45.524014,https://fullfact.org/online/new-york-synagogue...,"(ψ, ε, υ, δ, ή, ς)","(Έ, ν, α, ς, , ι, σ, χ, υ, ρ, ι, σ, μ, ό, ς, ..."
4,4,"(Τ, α, , ά, τ, ο, μ, α, , μ, ε, , λ, ι, γ, ...","(Τ, α, , κ, ρ, ι, τ, ή, ρ, ι, α, , ε, π, ι, ...",Charlotte Green,12 January 2024,20:31:45.524014,https://fullfact.org/online/49-pound-energy-vo...,"(Λ, ε, ί, π, ε, ι, , τ, ο, , π, λ, α, ί, σ, ...","(Μ, ι, α, , δ, η, μ, ο, σ, ί, ε, υ, σ, η, , ..."


In [53]:
save_df(df,'fullfacts_df.csv', None)

In [43]:
test = pd.read_csv('../data/df_fullfacts_')

In [45]:
test.shape

(1451, 9)

In [46]:
test = test[['id','claim','label']]
test

,id,claim,label
0,0,Ένα βίντεο δείχνει ένα βρετανικό πετρελαϊκό δε...,ψευδής
1,1,"Το Argos πωλεί κονσόλες PlayStation 5 για £ 3,...",ψευδής
2,2,Μια αναφορά που ζητά την απέλαση του ισραηλινο...,Λείπει το πλαίσιο
3,3,Μια σήραγγα που βρίσκεται κάτω από μια συναγωγ...,ψευδής
4,4,Τα άτομα με λιγότερο από £ 4 στον μετρητή προπ...,Λείπει το πλαίσιο
...,...,...,...
1446,1446,Ένα πορτοκαλί έχει δοκιμαστεί θετικά για το CO...,Λείπει το πλαίσιο
1447,1447,Το εμβόλιο AstraZeneca στο Ηνωμένο Βασίλειο πρ...,ψευδής
1448,1448,Ένας 12χρονος τραβήχτηκε από ξένους και επιτέθ...,ψευδής
1449,1449,Ο υπουργός Υγείας Nadine Dorries δήλωσε ότι οι...,Λείπει το πλαίσιο


In [47]:
test.label.value_counts()

ψευδής               901
Λείπει το πλαίσιο    237
εν μέρει ψευδής      225
αλλοιωμένος           50
σάτυρα                23
αληθής                14
ψεύτικος τίτλος        1
Name: label, dtype: int64

In [48]:
mapping = {'αληθής':1, 'ψευδής':0, 'Λείπει το πλαίσιο':2,'αλλοιωμένος':3,'σάτυρα':4,'εν μέρει ψευδής':0, '0':0,'1':1}
test['label'] = test['label'].map(mapping)
test['label'].value_counts()

0.0    1126
2.0     237
3.0      50
4.0      23
1.0      14
Name: label, dtype: int64

In [14]:
test = pd.read_csv('../data/id_claim_label_snopes.csv')

In [16]:
test.drop_duplicates(subset=['claim'], ignore_index=True, inplace=True
                    )

In [11]:
save_df(df2,'snopes2_df.csv', None)

In [12]:
test = pd.read_csv('../data/snopes2_df.csv')

In [13]:
test

,id,claim,article_title,author,publish_date,harvest_date,url,label,article_text
0,0,Η κεντρική τρύπα σε ένα κουτάλι σπαγγέτι έχει ...,Κεντρικές τρύπες σε κουτάλια σπαγγέτι που προο...,Madison Dapcevich,"Jan 20, 2024",21:33:40.992266,https://www.snopes.com/fact-check/holes-pasta-...,Mixture,Απαίτηση:\n\nΗ κεντρική τρύπα σε ένα κουτάλι σ...
1,1,Μια φωτογραφία δείχνει τον πρώην προπονητή της...,Η φωτογραφία δείχνει τον Bill Belichick που πα...,Jordan Liles,"Jan 19, 2024",21:33:40.992266,https://www.snopes.com/fact-check/belichick-ch...,Miscaptioned,Απαίτηση:\n\nΜια φωτογραφία δείχνει τον πρώην ...
2,2,Ένα βίντεο που δημοσιεύτηκε στο Instagram δείχ...,Είναι αυτό το πραγματικό βίντεο του αεροπλάνου...,Jack Izzo,"Jan 19, 2024",21:33:40.992266,https://www.snopes.com/fact-check/miami-plane-...,True,Απαίτηση:\n\nΈνα βίντεο που δημοσιεύτηκε στο I...
3,3,Μια εικόνα που μοιράστηκε τον Ιανουάριο του 20...,Είναι μια πραγματική εικόνα του Barron Trump;,Taija PerryCook,"Jan 19, 2024",21:33:40.992266,https://www.snopes.com/fact-check/barron-trump...,True,Απαίτηση:\n\nΜια εικόνα που μοιράστηκε τον Ιαν...
4,4,Οι ρωσικές αρχές ξεκινούν την κατασκευή ενός ε...,Είναι η Ρωσία ένα «μεταναστευτικό χωριό» για τ...,Alex Kasprak,"Jan 19, 2024",21:33:40.992266,https://www.snopes.com/fact-check/russian-cons...,Unproven,Απαίτηση:\n\nΟι ρωσικές αρχές ξεκινούν την κατ...
...,...,...,...,...,...,...,...,...,...
2949,2949,Ο Ιησούς και οι μαθητές του θα απεικονιστούν ω...,Θα απεικονιστεί ο Ιησούς ως ομοφυλόφιλος σε μι...,Barbara Mikkelson,"Apr 20, 2000",21:33:40.992266,https://www.snopes.com/fact-check/gay-jesus-film/,False,Απαίτηση:\n\nΟ Ιησούς και οι μαθητές του θα απ...
2950,2950,Το εμβόλιο HPV Gardasil έχει αποδειχθεί ότι έχ...,Επείγουσα προειδοποίηση για το gardasil,David Mikkelson,"Apr 22, 2009",21:33:40.992266,https://www.snopes.com/fact-check/on-gardasil/,False,Απαίτηση:\n\nΤο εμβόλιο HPV Gardasil έχει αποδ...
2951,2951,"Ως υπουργός Εξωτερικών, η Χίλαρι Κλίντον τεχνη...",Η Χίλαρι Κλίντον καταστέλλει τον ελάχιστο μισθ...,Brooke Binkowski,"Apr 5, 2016",21:33:40.992266,https://www.snopes.com/fact-check/hillary-clin...,Mixture,"Απαίτηση:\n\nΩς υπουργός Εξωτερικών, η Χίλαρι ..."
2952,2952,Οι αριθμητικοί κωδικοί PLU που χρησιμοποιούντα...,Οι κωδικοί PLU τεκμηριώνουν πώς καλλιεργούνται...,David Mikkelson,"May 30, 2012",21:33:40.992266,https://www.snopes.com/fact-check/plu-codes/,Mixture,Απαίτηση:\n\nΟι αριθμητικοί κωδικοί PLU που χρ...
